In [31]:
import tensorflow as tf

In [32]:
numDimensions = 200
maxSeqLength = 65
batchSize = 24
lstmUnits = 64
numClasses = 3
iterations = 100000

In [33]:
! ls

clean_tweet_subc_withuser_f.csv  RNN_suba_may_god_bless_me.ipynb
idsMatrix.npy			 test_clean_c.csv
model_final_subc.ipynb		 test_clean_withuser.csv
models				 Untitled.ipynb
output_rnn_subc.ipynb		 wordList-200.npy
RNN_100_subc.ipynb		 wordVectors-200.npy


In [34]:
import numpy as np
wordsList = np.load('wordList-200.npy').tolist()
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load('wordVectors-200.npy')

In [35]:
import tensorflow as tf
tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)

lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.25)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

In [36]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('models'))

INFO:tensorflow:Restoring parameters from models/pretrained_lstm.ckpt-40000


/home/preeti/.local/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [39]:
!ls

clean_tweet_subc_withuser_f.csv  RNN_suba_may_god_bless_me.ipynb
idsMatrix.npy			 test_clean_c.csv
model_final_subc.ipynb		 test_clean_withuser.csv
models				 Untitled.ipynb
output_rnn_subc.ipynb		 wordList-200.npy
RNN_100_subc.ipynb		 wordVectors-200.npy


In [49]:
import pandas as pd
import numpy as np

csv = 'test_clean_c.csv'
df = pd.read_csv(csv)
df.head()

,id,text
0,15923,democrats support antifa muslim brotherhood ms...
1,60133,unity demo to oppose the far right in enough i...
2,83681,what the fuck did he do this time
3,65507,user do you get the feeling he is kissing user...
4,34263,he is liar like the rest of the


In [50]:
len(df.text)

213

In [51]:
messages_test = df.text

In [52]:
# Removes punctuation, parentheses, question marks, etc., and leaves only alphanumeric characters
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

In [53]:
def test_id(l,u) :
  numFiles=24
  ids_test = np.zeros((numFiles, maxSeqLength), dtype='int32')
  messageCounter = 0
  for i in range(l,u):
    if messageCounter == 24:
      break
    indexCounter = 0
    cleanedLine = cleanSentences(str(messages_test[i]))
    split = cleanedLine.split()
    for word in split:
      try:
        ids_test[messageCounter][indexCounter] = wordsList.index(word)
      except ValueError:
        ids_test[messageCounter][indexCounter] = 399999
      indexCounter = indexCounter + 1
      if indexCounter >= maxSeqLength:
        break
    messageCounter = messageCounter + 1
  return (ids_test)

In [54]:
def res_gen(predsent):
  res = []
  for item in predsent :
    if (item[0] > item[1]) and  (item[0] > item[2]) :
      res.append(0)
    else :
        if (item[1] > item[0]) and  (item[1] > item[2]) :
            res.append(1)
        else :
            res.append(2)
  return(res) 

In [55]:
num = len(df.text)
itr = num // batchSize
res = []
for i in range(itr):
  l = i*24
  u = l+24
  ids = test_id(l,u)
  predictedSentiment = sess.run(prediction, {input_data: ids})
  res_itr = res_gen(predictedSentiment)
  res = res + res_itr
  
  

In [56]:
ids = test_id(u,len(df.text))
predictedSentiment = sess.run(prediction, {input_data: ids})
res_itr = res_gen(predictedSentiment)
res = res + res_itr




In [48]:
l

168

In [19]:
u

192

In [57]:
len(res)

216

In [58]:
finalres = res[:-3]

In [59]:
len(finalres)

213

In [25]:
df.head()

,id,text,output
0,15923,democrats support antifa muslim brotherhood ms...,0
1,60133,unity demo to oppose the far right in enough i...,0
2,83681,what the fuck did he do this time,1
3,65507,user do you get the feeling he is kissing user...,0
4,12588,user nigga ware da hits at,0


In [60]:
df['output'] = finalres

In [61]:
df.output.value_counts()

0    120
2     70
1     23
Name: output, dtype: int64

In [62]:
out_df = pd.DataFrame({'id':df.id, 'sentiment':df.output})
out_df.head()

,id,sentiment
0,15923,2
1,60133,2
2,83681,0
3,65507,2
4,34263,0


In [63]:
z = {0 : 'IND',1 :'OTH' , 2 : 'GRP'}
out_df['sentiment'] = out_df.sentiment.map(z)

In [64]:
out_df.head()

,id,sentiment
0,15923,GRP
1,60133,GRP
2,83681,IND
3,65507,GRP
4,34263,IND


In [65]:
out_df.to_csv('output_rnn_subc.csv',header=None ,index=False ,encoding='utf-8')